In [5]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
import tensorflow as tf

data = tf.keras.datasets.fashion_mnist

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation=tf.nn.relu),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax)
 ])

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=50, callbacks=[callbacks])

Epoch 1/50
1875/1875 [==============================] - 1s 606us/step - loss: 0.5038 - accuracy: 0.8235
Epoch 2/50
1875/1875 [==============================] - 1s 600us/step - loss: 0.3783 - accuracy: 0.8637
Epoch 3/50
1875/1875 [==============================] - 1s 653us/step - loss: 0.3403 - accuracy: 0.8768
Epoch 4/50
1875/1875 [==============================] - 1s 606us/step - loss: 0.3145 - accuracy: 0.8848
Epoch 5/50
1875/1875 [==============================] - 1s 595us/step - loss: 0.2965 - accuracy: 0.8909
Epoch 6/50
1875/1875 [==============================] - 1s 604us/step - loss: 0.2805 - accuracy: 0.8954
Epoch 7/50
1875/1875 [==============================] - 1s 601us/step - loss: 0.2685 - accuracy: 0.8998
Epoch 8/50
1875/1875 [==============================] - 1s 592us/step - loss: 0.2589 - accuracy: 0.9041
Epoch 9/50
1875/1875 [==============================] - 1s 596us/step - loss: 0.2492 - accuracy: 0.9067
Epoch 10/50
1875/1875 [==============================] - 1s 598u

In [8]:
l, a = model.evaluate(test_images, test_labels, verbose=0)

print(f'Loss: {l}')
print(f'Accuracy: {a}')

Loss: 0.40856632590293884
Accuracy: 0.8902000188827515


In [9]:
# Callback for different scenarios - more advanced

class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, accuracy_threshold=0.002, loss_threshold=0.002):
        super(CustomCallback, self).__init__()
        self.accuracy_threshold = accuracy_threshold
        self.loss_threshold = loss_threshold
        self.best_accuracy = 0
        self.best_loss = float('inf')
    
    def on_epoch_end(self, epoch, logs={}):
        current_accuracy = logs.get('accuracy')
        current_loss = logs.get('loss')
        
        if current_accuracy is None or current_loss is None:
            return
        
        accuracy_diff = current_accuracy - self.best_accuracy
        loss_diff = self.best_loss - current_loss
        
        if accuracy_diff <= self.accuracy_threshold and loss_diff <= self.loss_threshold:
            print("\nAccuracy and loss improvement below thresholds. Stopping training!")
            self.model.stop_training = True
        else:
            if current_accuracy > self.best_accuracy:
                self.best_accuracy = current_accuracy
            if current_loss < self.best_loss:
                self.best_loss = current_loss

    